# Train a customized model

In [3]:
import os
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow import keras
import glob
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Maybe you need to define a new model strucure, we can inherit class `BaseSRModel`.

Here we use `SRCNN` as example:

## Model establishing

In [4]:
from src.model import BaseSRModel

**It's noted that:**

- In original paper, the author use "valid" padding when training, yet "same" padding when testing. Here as an example, we set padding "valid" and thus the label size should be modified later. See http://mmlab.ie.cuhk.edu.hk/projects/SRCNN.html for details

- The `SRCNN` model pre-defined in module `models` use "same" padding by default.

- Learning rate of the last layer in origional paper is set to 1e-5, others are set to 1e-4. I don't know how to do different learning rate for different layers in tensorflow... So I set all to 1e-5 for convenience...

In [5]:
class SRCNN(BaseSRModel):
    """
    Using 9-1-5 model.
    """

    def __init__(self, scale, model_name, channel=1):

        super(SRCNN, self).__init__(scale, model_name, channel)
        
        # kernel size and number
        self.f1 = 9
        self.f2 = 1
        self.f3 = 5

        self.n1 = 64
        self.n2 = 32

    def create_model(self, load_weights=False, weights_path=None):

        inp = super(SRCNN, self).create_model()

        x = layers.Convolution2D(self.n1, (self.f1, self.f1),
                                 activation='relu', padding='valid', name='level1')(inp)
        x = layers.Convolution2D(self.n2, (self.f2, self.f2),
                                 activation='relu', padding='valid', name='level2')(x)

        out = layers.Convolution2D(self.channel, (self.f3, self.f3),
                                   padding='valid', name='output')(x)

        model = keras.Model(inp, out)

        if load_weights:
            weights_path = self.weights_path if weights_path is None else weights_path
            model.load_weights(self.weight_path)
            print("loaded model %s from %s" % (self.model_name, weights_path))

        self.model = model
        return self

    def lr_schedule(self, epoch, *args, **kwargs):
        return 1e-5

## Data Pipeline

**Pre-work is same as `train_models` notebook.**

In [6]:
train_dir = "./Image/set14" # Arbitrary
valid_dir = "./Image/set5"
AUTOTUNE = tf.data.experimental.AUTOTUNE
SCALE = 3

In [7]:
from src.write2tfrec import write_dst_tfrec, load_tfrecord

cache_dir = "./cache"
os.makedirs(cache_dir, exist_ok=True)

In [8]:
if not os.path.isfile("./cache/set14_train_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(train_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set14_train_48x48.tfrec")
    
if not os.path.isfile("./cache/set5_valid_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(valid_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set5_valid_48x48.tfrec")


**Preprocessing function here is a little different:**
    
- Lr-image should be upsample with `bicubic` kernel
- Considering the padding issue, we need to center-crop Hr-patch to the same size as output of the model.
- To follow the original paper, we need to transfer RGB image to Y channel only before training.

In [9]:
from src.preprocess import degrade_image
from src.data_utils import center_crop, rgb2ycbcr


def preprocess(hr):
    lr, hr = degrade_image(hr, SCALE, method=2, restore_shape=True)
    hr = center_crop(hr, (36, 36))
    # Using Y channel only
    lr, hr = [rgb2ycbcr(x)[...,0][...,tf.newaxis] for x in [lr, hr]]
    return lr, hr

In [10]:
trdst = load_tfrecord(48, "./cache/set14_train_48x48.tfrec").map(preprocess).repeat()
valdst = load_tfrecord(48, "./cache/set5_valid_48x48.tfrec").map(preprocess)

## Training

In [11]:
model = SRCNN(scale=SCALE, model_name="SRCNN",
                      channel=1).create_model(load_weights=False,
                                              weights_path=None)
model.fit(trdst,
          valdst,
          nb_epochs=2,
          steps_per_epoch=20,
          batch_size=16,
          use_wn=False)

Training model : SRCNN_X3


W0623 15:51:50.750764  4944 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/2
 1/20 [>.............................] - ETA: 45s - loss: 0.1226 - psnr_tf: 9.7831

W0623 15:51:53.370826  4944 callbacks.py:236] Method (on_train_batch_end) is slow compared to the batch update (0.162571). Check your callbacks.


15/20 [=====================>........] - ETA: 21s - loss: 0.1352 - psnr_tf: 9.46 - ETA: 5s - loss: 0.1369 - psnr_tf: 9.8954 - ETA: 2s - loss: 0.1430 - psnr_tf: 9.781 - ETA: 0s - loss: 0.1501 - psnr_tf: 9.5017
Epoch 00001: saving model to ./weights/SRCNN_X3.h5
20/20 [==============================] - 3s 165ms/step - loss: 0.1434 - psnr_tf: 9.7495 - val_loss: 0.1236 - val_psnr_tf: 10.4736
Epoch 2/2
18/20 [==========================>...] - ETA: 0s - loss: 0.0953 - psnr_tf: 10.64 - ETA: 0s - loss: 0.1382 - psnr_tf: 9.4463 - ETA: 0s - loss: 0.1284 - psnr_tf: 10.13 - ETA: 0s - loss: 0.1246 - psnr_tf: 10.31 - ETA: 0s - loss: 0.1243 - psnr_tf: 10.2765
Epoch 00002: saving model to ./weights/SRCNN_X3.h5
20/20 [==============================] - 0s 20ms/step - loss: 0.1224 - psnr_tf: 10.3050 - val_loss: 0.0996 - val_psnr_tf: 11.4229


**Model's Info**

In [14]:
model.model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
level1 (Conv2D)              (None, None, None, 64)    5248      
_________________________________________________________________
level2 (Conv2D)              (None, None, None, 32)    2080      
_________________________________________________________________
output (Conv2D)              (None, None, None, 1)     801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
